In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from warnings import simplefilter
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("../input/1st-feature-selection-lasso-xgboost/new_df.csv")
df

,visit_year,visit_week,sum_visitors,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,30,1,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,1,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,3,1,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,78,45,33,39,2,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,272,68,68,68,4,67,41,6,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,2017,16,76,17,3,10,7,0,0,0,...,6.025058,35.659397,139.696440,444,152,5,5,5,5,175.355837
42277,2017,16,140,36,12,20,7,0,0,0,...,2.742844,34.705362,135.510025,74,280,5,5,5,5,170.215387
42278,2017,16,92,31,7,15,6,0,0,0,...,5.318636,35.661777,139.704051,444,184,5,5,5,5,175.365828
42279,2017,16,110,28,1,15,7,0,0,0,...,4.541093,35.726232,139.723240,444,220,5,5,5,5,175.449472


In [3]:
df_col = df.describe()
df_col = df_col.drop(['sum_visitors'], axis=1)
#(include ='int64')
df_col.columns

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit', 'holiday_week_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'station_great_circle', 'restaurant_latitude',
       'restaurant_longitude', 'number_of_restaurants_in_area',
       'weekend_visitors', 'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude'],
      dtype='object')

In [4]:
varSel = pd.DataFrame({'Variable': df_col.columns})
varSel

,Variable
0,visit_year
1,visit_week
2,max_visitors
3,min_visitors
4,avg_visitors
5,count_visitors
6,sum_reserve_visitors
7,max_reserve_visitors
8,min_reserve_visitors
9,avg_reserve_visitors


In [5]:
nm = df_col.columns
nm = nm.append(pd.Index(['sum_visitors']))
nm

Index(['visit_year', 'visit_week', 'max_visitors', 'min_visitors',
       'avg_visitors', 'count_visitors', 'sum_reserve_visitors',
       'max_reserve_visitors', 'min_reserve_visitors', 'avg_reserve_visitors',
       'count_reserve_visitors', 'holiday_flg_visit', 'holiday_week_visit',
       'number_of_holiday_days_visit', 'area_id', 'genre_id', 'station_id',
       'station_vincenty', 'station_great_circle', 'restaurant_latitude',
       'restaurant_longitude', 'number_of_restaurants_in_area',
       'weekend_visitors', 'sum_diff_days_between_reserve_visit_category',
       'max_diff_days_between_reserve_visit_category',
       'avg_diff_days_between_reserve_visit_category',
       'min_diff_days_between_reserve_visit_category', 'latitude_longitude',
       'sum_visitors'],
      dtype='object')

In [6]:
df = df[nm].copy()
df.head()

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude,sum_visitors
0,2016,1,30,30,30,1,0,0,0,0,...,34.695124,135.197853,57,30,5,5,5,5,169.892977,30
1,2016,1,1,1,1,1,0,0,0,0,...,35.661777,139.704051,444,1,5,5,5,5,175.365828,1
2,2016,1,3,3,3,1,0,0,0,0,...,35.693840,139.703549,444,3,5,5,5,5,175.397390,3
3,2016,1,45,33,39,2,0,0,0,0,...,35.693840,139.703549,444,78,5,5,5,5,175.397390,78
4,2016,1,68,68,68,4,67,41,6,16,...,33.319286,130.508374,127,272,1,1,1,1,163.827660,272


In [7]:
X = df.iloc[:,:-1]
X.head()

,visit_year,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,sum_reserve_visitors,max_reserve_visitors,min_reserve_visitors,avg_reserve_visitors,...,station_great_circle,restaurant_latitude,restaurant_longitude,number_of_restaurants_in_area,weekend_visitors,sum_diff_days_between_reserve_visit_category,max_diff_days_between_reserve_visit_category,avg_diff_days_between_reserve_visit_category,min_diff_days_between_reserve_visit_category,latitude_longitude
0,2016,1,30,30,30,1,0,0,0,0,...,1.274882,34.695124,135.197853,57,30,5,5,5,5,169.892977
1,2016,1,1,1,1,1,0,0,0,0,...,5.318636,35.661777,139.704051,444,1,5,5,5,5,175.365828
2,2016,1,3,3,3,1,0,0,0,0,...,4.202990,35.693840,139.703549,444,3,5,5,5,5,175.397390
3,2016,1,45,33,39,2,0,0,0,0,...,4.202990,35.693840,139.703549,444,78,5,5,5,5,175.397390
4,2016,1,68,68,68,4,67,41,6,16,...,2.258974,33.319286,130.508374,127,272,1,1,1,1,163.827660


In [8]:
y = df.iloc[:,-1:]
y.head()

,sum_visitors
0,30
1,1
2,3
3,78
4,272


### Variable Selection using Random Forest

In [9]:
rfmod = RandomForestClassifier(max_depth=16).fit(X, y.values.ravel())
#(max_depth=16)
#rfmod.feature_importances_

In [10]:
model = SelectFromModel(rfmod, prefit=True)
model.get_support()

array([False,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False])

In [11]:
varSel['RandomForest'] = model.get_support()
#.astype('int64')
varSel

,Variable,RandomForest
0,visit_year,False
1,visit_week,True
2,max_visitors,True
3,min_visitors,True
4,avg_visitors,True
5,count_visitors,True
6,sum_reserve_visitors,False
7,max_reserve_visitors,False
8,min_reserve_visitors,False
9,avg_reserve_visitors,False


### Summarization and Selection of Variables 

In [12]:
varSel['Sum'] = np.sum(varSel,axis=1)
varSel

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:85: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Variable,RandomForest,Sum
0,visit_year,False,0
1,visit_week,True,1
2,max_visitors,True,1
3,min_visitors,True,1
4,avg_visitors,True,1
5,count_visitors,True,1
6,sum_reserve_visitors,False,0
7,max_reserve_visitors,False,0
8,min_reserve_visitors,False,0
9,avg_reserve_visitors,False,0


In [13]:
varSel.groupby('Sum')['Variable'].count()

Sum
0    22
1     6
Name: Variable, dtype: int64

In [14]:
# We can now decide a threshold for selecting our variables!

In [15]:
varSel[varSel['Sum']>=1]

,Variable,RandomForest,Sum
1,visit_week,True,1
2,max_visitors,True,1
3,min_visitors,True,1
4,avg_visitors,True,1
5,count_visitors,True,1
22,weekend_visitors,True,1


In [16]:
variables = varSel[varSel['Sum']>=1].iloc[:, [0]]
variables

,Variable
1,visit_week
2,max_visitors
3,min_visitors
4,avg_visitors
5,count_visitors
22,weekend_visitors


In [17]:
variables_to_columns = variables.transpose().iloc[:].head()
df_variables_to_columns = pd.DataFrame(variables_to_columns)
sum_visitors = ['sum_visitors']
df_variables_to_columns['sum_visitors'] = sum_visitors
df_variables_to_columns.to_csv('third_variables_data.csv',index = False)
df_variables_to_columns

,1,2,3,4,5,22,sum_visitors
Variable,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,weekend_visitors,sum_visitors


In [18]:
top_columns = pd.read_csv("./third_variables_data.csv",header = 1).columns
top_columns

Index(['visit_week', 'max_visitors', 'min_visitors', 'avg_visitors',
       'count_visitors', 'weekend_visitors', 'sum_visitors'],
      dtype='object')

In [19]:
df_After_Feature_Selection = pd.DataFrame(df[top_columns])
df_After_Feature_Selection.to_csv("./Third_Feature_Selection.csv",index = False)
df_After_Feature_Selection

,visit_week,max_visitors,min_visitors,avg_visitors,count_visitors,weekend_visitors,sum_visitors
0,1,30,30,30,1,30,30
1,1,1,1,1,1,1,1
2,1,3,3,3,1,3,3
3,1,45,33,39,2,78,78
4,1,68,68,68,4,272,272
...,...,...,...,...,...,...,...
42276,16,17,3,10,7,152,76
42277,16,36,12,20,7,280,140
42278,16,31,7,15,6,184,92
42279,16,28,1,15,7,220,110
